# Import packages

In [24]:
import requests
import json
import pandas as pd
import numpy as np
from io import StringIO
from bs4 import BeautifulSoup as bs

from datetime import date
import covidcast

import geopandas as gpd

pd.set_option('display.max_columns', 100)

In [25]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
s = requests.get(url).text
nymask = pd.read_csv(StringIO(s))

In [26]:
nymask.COUNTYFP = nymask.COUNTYFP.astype(str)
nymask.COUNTYFP = np.where(nymask['COUNTYFP'].str.len() == 4, '0' + nymask.COUNTYFP, nymask.COUNTYFP) 

In [27]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
s = requests.get(url).text
df = pd.read_csv(StringIO(s))
print(df.shape)
print(df.columns)
# df6.head()

(897801, 6)
Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')


In [28]:
mask_ind = covidcast.signal("fb-survey", "smoothed_wearing_mask",
                        date(2020, 10, 1), date(2020, 11, 4),
                        "county")

In [29]:
mask_fip = mask_ind.loc[~mask_ind.geo_value.str.endswith('000')]

In [30]:
print(mask_ind.geo_value.value_counts().shape)
mask_fip.geo_value.value_counts().shape

(640,)


(592,)

In [31]:
mask_ind_means = pd.DataFrame(mask_fip.groupby(['geo_value'])['value'].mean()).reset_index()
mask_ind_means.columns = ['FIPS', 'ind_mask']
mask_ind_means

,FIPS,ind_mask
0,01003,75.669396
1,01069,84.805330
2,01073,88.936083
3,01089,89.685856
4,01097,82.273652
...,...,...
587,55133,85.785520
588,55139,88.268850
589,55141,82.577264
590,56021,72.517789


In [32]:
election = pd.read_csv('data/county_statistics.csv', index_col="Unnamed: 0")

In [33]:
election = election[election.county != 'Unassigned']
election['county'] = election['county'] + ', ' + election['state']
# election.drop(['state'], axis=1, inplace=True)

In [34]:
to_keep = ['county', 'total_votes20' , 'percentage20_Joe_Biden']
election = election.dropna(subset=['votes20_Joe_Biden'])

In [35]:
merge_elec = election[to_keep]

In [36]:
print(merge_elec.shape)
# election.columns

(4633, 3)


In [39]:
geo = gpd.read_file('https://opendata.arcgis.com/datasets/4cb598ae041348fb92270f102a6783cb_0.geojson')


In [45]:
geo = geo[~geo.Countyname.str.contains("Out of")]
geo = geo[~geo.Countyname.str.contains("Unassigned")]
geo = geo[~geo.ST_Name.str.contains("Puerto Rico")]
geo.drop(geo.tail(7).index, inplace=True)
geo['county'] = geo['Countyname'] + ', ' + geo['ST_Abbr']
geo.drop(columns=['Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 
                     'Day_6', 'Day_7', 'Day_8', 'Day_9', 'Day_10', 
                     'Day_11', 'Day_12', 'Day_13', 'Day_14', 
                     'OBJECTID', 'ST_ID', 'PCTPOVALL_', 'Med_HH_Inc', 
                     'State_Fata', 'DateChecke', 'url', 'Thumbnail', 
                     'State_Conf', 'State_Deat', 'State_Reco', 'State_Test', 
                     'TotalPop', 'NonHispWhP', 'BlackPop', 'AmIndop', 'PacIslPop', 
                     'OtherPop', 'TwoMorPop', 'HispPop', 'NonHisp', 'Age_85', 
                     'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 
                     'Agetotal', 'AsianPop', 'Countyname', 'ST_Name'], 
            inplace=True)


In [41]:
geo['Staf_Bed_Perc'] = geo.Beds_Staff / geo.Beds_Licen
geo['Aged_Perc'] = geo.AgedPop / geo.POP_ESTIMA
geo['Pov_Perc'] = geo.POVALL_201 / geo.POP_ESTIMA
geo['Aged_Per_ICU'] = geo.AgedPop / geo.Beds_ICU
geo.drop(columns=['Age_Less15', 'Age_15_24', 'Age_25_34', 'Age_35_64', 'Age_65_74', 'Confirmed', 'Deaths'], inplace=True)
geo.drop(columns=['Unemployed', 'Beds_Licen', 'Beds_Staff', 'Ventilator', 'Unemployed', 'Beds_ICU', 'AgedPop'], inplace=True)
geo.drop(columns=['Wh_Alone', 'Bk_Alone', 'AI_Alone', 'As_Alone', 'NH_Alone', 'SO_Alone', 'Two_More', 'Not_Hisp', 'Age_Over75'], inplace=True)

In [42]:
geo.drop(columns=['Recovered', 'Active'], inplace=True)
geo.drop(columns=['EM_notes'], inplace=True)
geo.EM_date = geo.EM_date.str.extract(r'((\d+)\/(\d{2})\/(\d{4}))')
geo.EM_type = geo.EM_type.replace({'Govt Ordered Community Quarantine': 'CQ', 'Govt Directed Social Distancing': 'SD'})

In [43]:
geo.shape

(3142, 70)

In [46]:
geo.columns

Index(['ST_Abbr', 'FIPS', 'FatalityRa', 'Confirmedb', 'DeathsbyPo',
       'Unemployme', 'EM_type', 'EM_date', 'Beds_ICU', 'POP_ESTIMA',
       'POVALL_201', 'Median_Hou', 'AgedPop', 'NewCases', 'NewDeaths',
       'NewCasebyP', 'Inpat_Occ', 'ICU_Occ', 'Shape__Area', 'Shape__Length',
       'geometry', 'county', 'Staf_Bed_Perc', 'Aged_Perc', 'Pov_Perc',
       'Aged_Per_ICU'],
      dtype='object')

In [47]:
elec_geo_df = geo.merge(merge_elec, on = 'county',how='left', sort=True)
elec_geo_df = elec_geo_df.merge(mask_ind_means, how = 'left', on = 'FIPS')
elec_geo_df.percentage20_Joe_Biden.isna().sum()

94

In [48]:
elec_geo_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3135 entries, 0 to 3134
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   ST_Abbr                 3135 non-null   object  
 1   FIPS                    3135 non-null   object  
 2   FatalityRa              3135 non-null   float64 
 3   Confirmedb              3135 non-null   float64 
 4   DeathsbyPo              3135 non-null   float64 
 5   Unemployme              3135 non-null   float64 
 6   EM_type                 3135 non-null   object  
 7   EM_date                 2510 non-null   object  
 8   Beds_ICU                3135 non-null   int64   
 9   POP_ESTIMA              3135 non-null   int64   
 10  POVALL_201              3135 non-null   int64   
 11  Median_Hou              3135 non-null   int64   
 12  AgedPop                 3135 non-null   int64   
 13  NewCases                3135 non-null   int64   
 14  NewDeaths       

In [49]:
elec_geo_df.to_file('data/elec.geojson', driver='GeoJSON')